In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
df = pd.read_csv('C:\\Users\\M\\Machine_L\\data\\loan_data.csv')
df.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0


### 대출을 허용할지 말지를 분류/ 예측하는 머신러닝 모델 생성
- 새로운 고객에 대해서 대출여부를 판단할 수 있도록 하는데 사용
- 컬럼명 참고
- credit.policy: 신용정책?
- purpose: 대출목적
- int.rate : 대출 이자율
- installment : 할부금
- log.annual.inc : 연수입
- dti : 총부채상환비율
- fico : 개인의 신용 점수(미국ver.)
- days.with.cr.line: 신용카드 사용기간
- revol.bal : 신용카드 잔여한도
- revol.util : 신용카드 사용자의 신용카드 사용률
- nq.last.6mths : 지난 6개월 간의 신규 대출 신청 건수
- delinq.2yrs : 최근 2년간의 연체 여부
- pub.rec : 개인의 공개된 신용 보고서?
- not.fully.paid 컬럼은 다 갚았는지 안갚았는지(1이면 대출을 갚지 못한 case)

In [19]:
#1. 전처리 - 문자열 인코딩, onehot
df_ohe = pd.get_dummies(df, dtype = 'int')

In [21]:
#2. X,y 설정
X = df_ohe.drop(columns = ['not.fully.paid'])
y = df_ohe['not.fully.paid']

In [22]:
#3. train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2024)

In [23]:
#4. 모델 학습
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [24]:
#5. 예측
pred = lr.predict(X_test)

In [25]:
#6. 모델평가
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.84      1.00      0.91      2417
           1       0.50      0.00      0.00       457

    accuracy                           0.84      2874
   macro avg       0.67      0.50      0.46      2874
weighted avg       0.79      0.84      0.77      2874



## 결과해석
못갚을 사람을 잡아내야함 => 결과가 1인 사람\
accuracy는 높지만(84%) 1에 대한 결과는 0.5, 0으로 안좋은 편이라\
못갚을 사람을 잡아내는데 쓸 수 있는가? => X

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
cm = confusion_matrix(y_test, pred)
ac = accuracy_score(y_test, pred)
ps = precision_score(y_test, pred)
rs = recall_score(y_test, pred)
fs = f1_score(y_test, pred)

print('accuracy_score : ', ac)
print('precision_score : ', ps)
print('recall_score : ', rs)
print('f1_score : ', fs)

accuracy_score :  0.8409881697981907
precision_score :  0.5
recall_score :  0.002188183807439825
f1_score :  0.004357298474945534


In [35]:
from sklearn.metrics import roc_auc_score
proba = lr.predict_proba(X_test)
roc_auc_score(y_test, proba[:, :1])

0.4127845340580806

In [31]:
proba

array([[0.81302879, 0.18697121],
       [0.79347575, 0.20652425],
       [0.88445911, 0.11554089],
       ...,
       [0.88400435, 0.11599565],
       [0.85641105, 0.14358895],
       [0.79700197, 0.20299803]])

In [36]:
#imbalanced data => smote